In [35]:
from gensim.models import KeyedVectors
import pandas as pd
import csv
import random

In [36]:
wv_from_text = KeyedVectors.load_word2vec_format("./toastynews.vec", binary=False)
wv = wv_from_text

In [37]:
# get pos 
# dfPos = pd.read_excel('qual_trnslSelection_RIGHT ORDER.xlsx')

# with open('wordpos-1689192352.csv', newline='', encoding="utf-8") as posfile:
#       tmp = csv.reader(posfile, delimiter=',')
#       posrow = list(tmp)

# wordlist = dfPos['TARGET_TRANSLATION'].tolist()

# words = []
# for i in range(len(posrow)):
#     words.append(posrow[i][0])
    
# # Loop through the Cantonese_Filtered column
# for index, word in enumerate(wordlist):
#     if not word or (word not in words):
#         dfPos.at[index, 'pos'] = 'NOTIN'
#     else:
#         pos = []
#         if words.count(word) > 1:
#             for i in range(len(words)):
#                 if words[i] == word:
#                     tmp = posrow[i][1].strip('[\']')
#                     [pos.append(i.strip(' \'')) for i in tmp.split(',') if i not in pos]
#         else:
#             tmp = posrow[words.index(word)][1].strip('[\'],')
#             [pos.append(i.strip(' \'')) for i in tmp.split(',')]
            
#         [i.strip('\' ') for i in pos]
#         dfPos.at[index, 'pos'] = sorted(pos)

# # Save the DataFrame to a new excel file
# dfPos.to_excel('qual_pos.xlsx', index=False)

In [38]:
# generate semantic related foil
dfTrans = pd.read_excel('qual_trnslSelection_RIGHT ORDER.xlsx')
# dfTrans = pd.read_excel('qual_pos.xlsx')

# Create a new column called "corr_cantonese"

poslist = dfTrans['pos'].tolist()
wordlist = dfTrans['cantonese_orig'].tolist()
targetlist = dfTrans['TARGET'].tolist()

curpos = []
posindex = 0
# Loop through the Cantonese_Filtered column
for index, word in enumerate(wordlist):
    if poslist[index] == 'NOTIN' or poslist[index] == 'EXCLUDED':
        continue
    if poslist[index] != curpos:
        curpos = poslist[index]
        posindex = index
        
    # Check if word is empty or not in vocab
    if not word or (word not in wv.key_to_index):
        dfTrans.at[index, 'SEMREL_FOIL_TRANSLATION'] = 'NOTIN'
        # print(f"{word} is not in vocab")
        # dfTrans.at[index, 'cantonese_map_furthest'] = 'NOTIN'
    else:
        # Find the most similar word in the wordlist
        similarityList = []
        similarityWordList = []
        
        for innerword in wordlist:
            if innerword == word: 
                continue
            if not innerword or (innerword not in wv.key_to_index): 
                continue
            if poslist[wordlist.index(innerword)] != curpos:
                continue
            
            similarity = wv.similarity(word, innerword)
            
            similarityList.append(similarity)
            similarityList.sort(reverse=True)
            idx = similarityList.index(similarity)
            similarityWordList.insert(idx, innerword)   
            
            f = open(f'words/{word}.txt', "w", encoding="utf-8")
            # f.write(most_similar_word)
            f.write(f"{word} {poslist[index]}")
            f.write(word)
            f.write("\n")
            for i in range(len(similarityList)):
                f.write(str(similarityWordList[i]))
                f.write(" ")
                f.write(str(similarityList[i]))
                f.write(" ")
                f.write(poslist[wordlist.index(innerword)])
                if(i==0 or i==2 or i==4 or i==9 or i==len(similarityList)-1):
                    f.write(f" <-- {i+1}")
                f.write("\n")
            f.close()
            
                 
        # Fill in the corr_cantonese column with the most similar word found
        # dfTrans.at[index, '1st'] = similarityWordList[0]
        # dfTrans.at[index, 'mapped_NewFileName'] = filenamelist[index]
        # dfTrans.at[index, '3rd'] = similarityWordList[2]
        if len(similarityWordList) > 4:
            dfTrans.at[index, 'SEMREL_FOIL'] = targetlist[wordlist.index(similarityWordList[4])]
            dfTrans.at[index, 'SEMREL_FOIL_TRANSLATION'] = similarityWordList[4]
            # dfTrans.at[index, '5th_pos'] = poslist[wordlist.index(similarityWordList[4])]
        else:
            # dfTrans.at[index, 'SEMREL_FOIL'] = len(similarityWordList)+1
            dfTrans.at[index, 'SEMREL_FOIL_TRANSLATION'] = "EXCLUDED"

# Save the DataFrame to a new excel file
dfTrans.to_excel('qual_trnslSelection.xlsx', index=False)

In [39]:
# get lists of all words with the same pos, store in dict
dfPos = pd.read_excel('qual_trnslSelection.xlsx')

poslist = dfPos['pos'].tolist()
wordlist = dfPos['TARGET_TRANSLATION'].tolist()
posdict = {}

# Loop through the TARGET_TRANSLATION column
for index, word in enumerate(wordlist):
    if poslist[index] == 'NOTIN' or poslist[index] == 'EXCLUDED':
        continue
    if poslist[index] not in posdict:
        posdict[poslist[index]] = []    
    
    if word not in posdict[poslist[index]]:
        posdict[poslist[index]].append(word)
        
print(posdict)

{"['代詞']": ['哪個', '邊度', '其他', '我嘅', '為甚麽', '嗰個', '什麼/甚麼', '自己', '誰', '幾多/多少/幾個', '為什麼/點解'], "['副詞']": ['一些', '最後', '可能', '多啲', '幾好', '唔好', '經常', '間中', '以前', '不能', '最初', '立即', '兩次', '好多次', '最近', '仍然/依然', '本來', '忽然', '暫時', '永遠', '尚未', '差不多'], "['動詞', '名詞']": ['頭痛', '警告', '嘗試', '遊行', '體驗', '決定', '旅行', '溝通', '想像', '比賽', '檢查', '生活', '挑戰', '電郵', '研究', '製作', '要求', '運動', '感覺', '懲罰', '解釋', '同意', '誤解', '笑', '命令', '保護', '尊重', '祈禱', '介紹', '進步', '退步', '失敗', '傳真', '交流', '設計', '安排', '集合', '測驗/考試', '訓練/培訓', '表演', '依靠', '翻譯', '唇讀', '發音', '影響', '津貼', '約會', '病', '習慣', '大便', '小便', '感冒', '攝影', '燒烤', '活動'], "['動詞', '形容詞']": ['悶', '嬲', '贏', '衰', '明白', '輸', '平衡', '成功', '辛苦', '清潔', '討厭', '緊張', '清楚', '投入', '得閒'], "['動詞', '語句']": ['傳訊息', '使灰心', '踩滑板', '碌落去', '遊船河', '聽音樂', '玩電子遊戲', '講大話', '拍馬屁', '給我', '最鍾意', '再來'], "['動詞']": ['開始', '疑惑', '沖涼', '打獵', '怕醜', '望住', '學習', '影印', '烤焗', '懷疑', '分解', '訂婚', '食', '類似', '飲', '等', '吞', '恭喜', '減', '身穿', '扮演', '投票', '成長', '放大', '跳舞', '看似', '諗', '離開', '攞', '吹牛', '跑', '剃', '設定（店鋪）

In [40]:
# change words that were modified for generating semantic foil to the original word
dfPos = pd.read_excel('qual_trnslSelection.xlsx')

semanticfoillist = dfPos['SEMREL_FOIL_TRANSLATION'].tolist()
cantonese_orig = dfPos['cantonese_orig'].tolist()
target_translation = dfPos['TARGET_TRANSLATION'].tolist()
f = open(f'change.txt', "w", encoding="utf-8")

for index, word in enumerate(cantonese_orig):
    if poslist[index] == 'EXCLUDED' or semanticfoillist[index] == 'NOTIN' or semanticfoillist[index] == 'EXCLUDED':
        continue
    if(semanticfoillist[index] not in target_translation):
        wordIdx = cantonese_orig.index(semanticfoillist[index])
        print(word, semanticfoillist[index], target_translation[wordIdx])
        f.write(word)
        f.write(semanticfoillist[index])
        f.write(target_translation[wordIdx])
        f.write("\n")
        # for i in range(len(similarityList)):
        #     f.write(str(similarityWordList[i]))
        #     f.write(" ")
        #     f.write(str(similarityList[i]))
        #     f.write(" ")
        #     f.write(poslist[wordlist.index(innerword)])
        #     if(i==0 or i==2 or i==4 or i==9 or i==len(similarityList)-1):
        #         f.write(f" <-- {i+1}")
        #     f.write("\n")
        semanticfoillist[index] = target_translation[wordIdx]
        dfPos['SEMREL_FOIL_TRANSLATION'] = semanticfoillist
f.close()
dfPos.to_excel('qual_trnslSelection.xlsx', index=False)

哪個 為什麼 為什麼/點解
其他 那個 嗰個
點解 多少 幾多/多少/幾個
什麼 那個 嗰個
自己 多少 幾多/多少/幾個
誰 那個 嗰個
多少 那個 嗰個
為什麼 多少 幾多/多少/幾個
間中 多次 好多次
體驗 訓練 訓練/培訓
旅行 考試 測驗/考試
研究 訓練 訓練/培訓
表演 訓練 訓練/培訓
讀唇 考試 測驗/考試
津貼 考試 測驗/考試
影印 錄影 錄影 (雙手)
分解 設定 設定（軟體）
吞 掉 掉低
上載 錄影 錄影 (雙手)
擔心 不介意 唔介意
量度 設定 設定（軟體）
下載 設定 設定（軟體）
影印 錄影 錄影 (雙手)
愛惜 同情 同情、可憐
堅持 不介意 唔介意
探熱 烘烤 烤焗
害怕 不介意 唔介意
開玩笑 不介意 唔介意
謝謝 不介意 唔介意
紅色 棕色 啡色
橙色 棕色 啡色
鎖匙 電話 視訊電話
相機 單反 有鏡頭的相機
表妹 堂姐 堂家姐
英文 字幕 說明文字、字幕
速遞 電話 視訊電話
速遞 電話 視訊電話
朋友 家人 屋企人
迪士尼樂園 郵輪 渡海小輪
不高興 不好吃 不好味
新的 大膽 臉皮厚
麻煩 不舒服 唔舒服
肚餓 睏 眼睏
麻煩 不舒服 唔舒服
糊塗 真的 真嘅


In [41]:
# get 20 randomised words for each item
# dfTrans = pd.read_csv('qual_trnslSelection.csv', encoding='utf-8-sig')
dfTrans = pd.read_excel('qual_trnslSelection.xlsx')
# dfTrans = pd.read_excel('HKSL_added.xlsx')

# dfTrans.insert(9,'5th_EntryID','')

semanticfoillist = dfTrans['SEMREL_FOIL_TRANSLATION'].tolist()
# filenamelist = dfTrans['NewFileName'].tolist()
poslist = dfTrans['pos'].tolist()
wordlist = dfTrans['TARGET_TRANSLATION'].tolist()
target = dfTrans['TARGET'].tolist()

curpos = []
posindex = 0
# print(posdict)
# Loop through the Cantonese_Filtered column
for index, word in enumerate(wordlist):
    if poslist[index] == 'EXCLUDED' or semanticfoillist[index] == 'NOTIN' or semanticfoillist[index] == 'EXCLUDED':
        continue
    # if poslist[index] != curpos:
    #     curpos = poslist[index]
    #     posindex = index
    
    listofpos = posdict[poslist[index]].copy()
    randomfoillist = []
    # print(word, semanticfoillist[index], len(listofpos))
    # print(poslist[index])
    # print(listofpos)
    print(word, semanticfoillist[index], poslist[index])
    listofpos.remove(word)
    # print(listofpos)
    listofpos.remove(semanticfoillist[index])
    # print(word, semanticfoillist[index], poslist[index])
    # print(listofpos == posdict[poslist[index]]) 
    if len(listofpos) < 18:
        tmp = random.sample(listofpos, 9)
        for i in tmp:
            randomfoillist.append(i)
            randomfoillist.append(i)
        # print(word, semanticfoillist[index], poslist[index], randomfoillist, listofpos, tmp)
    else:    
        randomfoillist = random.sample(listofpos, 18)
    
    # print(word, semanticfoillist[index], poslist[index], randomfoillist, listofpos)
    for i in range(1, 19):
        dfTrans.at[index, 'DUMMY_FOIL_' + str(i) + '_TRANSLATION'] = randomfoillist[i-1]
        dfTrans.at[index, 'DUMMY_FOIL_' + str(i)] = target[wordlist.index(randomfoillist[i-1])]
    # dfTrans.at[index, 'DUMMY_FOIL_1'] = randomfoillist[4]
    
    # dfTrans.at[index, '5th_EntryID'] = target[wordlist.index(semanticfoillist[index])]
    listofpos.clear()
    # randomfoillist.clear()

# dfTrans.to_excel('HKSL_added.xlsx', index=False)
# dfTrans.to_csv('qual_trnslSelection.csv', index=False)
dfTrans.to_excel('qual_trnslSelection.xlsx', index=False)

哪個 為什麼/點解 ['代詞']
邊度 自己 ['代詞']
其他 嗰個 ['代詞']
為甚麽 幾多/多少/幾個 ['代詞']
嗰個 自己 ['代詞']
什麼/甚麼 嗰個 ['代詞']
自己 幾多/多少/幾個 ['代詞']
誰 嗰個 ['代詞']
幾多/多少/幾個 嗰個 ['代詞']
為什麼/點解 幾多/多少/幾個 ['代詞']
一些 本來 ['副詞']
最後 兩次 ['副詞']
可能 永遠 ['副詞']
多啲 本來 ['副詞']
幾好 間中 ['副詞']
唔好 永遠 ['副詞']
經常 以前 ['副詞']
間中 好多次 ['副詞']
以前 可能 ['副詞']
不能 尚未 ['副詞']
最初 最後 ['副詞']
最後 兩次 ['副詞']
立即 最後 ['副詞']
兩次 最近 ['副詞']
好多次 最後 ['副詞']
最近 最後 ['副詞']
仍然/依然 本來 ['副詞']
本來 最後 ['副詞']
忽然 最後 ['副詞']
暫時 尚未 ['副詞']
永遠 最後 ['副詞']
尚未 最後 ['副詞']
差不多 兩次 ['副詞']
頭痛 小便 ['動詞', '名詞']
警告 電郵 ['動詞', '名詞']
嘗試 解釋 ['動詞', '名詞']
嘗試 解釋 ['動詞', '名詞']
遊行 祈禱 ['動詞', '名詞']
體驗 訓練/培訓 ['動詞', '名詞']
決定 解釋 ['動詞', '名詞']
旅行 測驗/考試 ['動詞', '名詞']
溝通 嘗試 ['動詞', '名詞']
想像 嘗試 ['動詞', '名詞']
比賽 運動 ['動詞', '名詞']
檢查 要求 ['動詞', '名詞']
生活 進步 ['動詞', '名詞']
挑戰 運動 ['動詞', '名詞']
電郵 約會 ['動詞', '名詞']
研究 訓練/培訓 ['動詞', '名詞']
製作 介紹 ['動詞', '名詞']
要求 同意 ['動詞', '名詞']
運動 進步 ['動詞', '名詞']
感覺 生活 ['動詞', '名詞']
懲罰 挑戰 ['動詞', '名詞']
解釋 介紹 ['動詞', '名詞']
同意 溝通 ['動詞', '名詞']
誤解 同意 ['動詞', '名詞']
笑 頭痛 ['動詞', '名詞']
運動 進步 ['動詞', '名詞']
命令 同意 ['動詞', '名詞']
保護 挑戰 ['動